In [1]:
import io
import os
import pickle

from PIL import Image, ImageOps
import numpy as np
from matplotlib import pyplot as plt

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import make_pipeline

from sklearn.multiclass import OneVsOneClassifier
from sklearn.cluster import KMeans


from sklearn.metrics import accuracy_score, f1_score, adjusted_rand_score, confusion_matrix, ConfusionMatrixDisplay

import pandas as pd
from IPython.display import display

plt.ion()
plt.show()

### Dataset bootstrap

In [2]:

# Load the dataset
raw_dataset = np.load("dataset_food101tiny.zip", allow_pickle=True)

dataset = {
    "train": {
        "data": [],
        "names": [],
        "labels": [],
        "unique_labels": [],
    },
    "valid": {
        "data": [],
        "names": [],
        "labels": [],
        "unique_labels": [],
    },
}

images_shape = (224, 224)

# For each image we have the path from which we extract the name and the label of the image
for dsKey in raw_dataset.keys():
    splittedKey = dsKey.split("/")

    img_type = splittedKey[2]
    img_label = splittedKey[3]
    img_name = splittedKey[4]

    img = Image.open(io.BytesIO(raw_dataset[dsKey]))
    img = ImageOps.fit(img, images_shape, Image.Resampling.LANCZOS).convert("RGB")

    img_array = np.asarray(img)

    dataset[img_type]["data"].append(img_array)
    dataset[img_type]["names"].append(img_name)
    dataset[img_type]["labels"].append(img_label)

dataset["train"]["data"] = np.reshape(np.asarray(dataset["train"]["data"]).mean(axis=3), (len(dataset["train"]['labels']), -1))

stdScaler = StandardScaler()
stdScaler.fit(dataset["train"]["data"])

dataset["train"]["data"] = stdScaler.transform(dataset["train"]["data"])
dataset["train"]["names"] = np.asarray(dataset["train"]["names"])

dataset["train"]["unique_labels"], dataset["train"]["labels"] = np.unique(
    np.asarray(dataset["train"]["labels"]), return_inverse=True
)

dataset["valid"]["data"] = np.reshape(np.asarray(dataset["valid"]["data"]).mean(axis=3), (len(dataset["valid"]['labels']), -1))
dataset["valid"]["data"] = stdScaler.transform(dataset["valid"]["data"])
dataset["valid"]["names"] = np.asarray(dataset["valid"]["names"])

dataset["valid"]["unique_labels"], dataset["valid"]["labels"] = np.unique(
    np.asarray(dataset["valid"]["labels"]), return_inverse=True
)


### Feature extraction using PCA and LDA

In [3]:
# Each method is tested with a set of numer of components
n_components_to_test = {
    "PCA": [3, 10, 50, 100, 200, 500, 1200],
    "LDA": [3, 5, 7, 9],
    "TSNE": [2, 3],
}

In [4]:
# Results to compare the methods the number of component changes
results_PCA = []
results_LDA = []

PCAs_instances = {}
LDAs_instances = {}
TSNEs_instances = {}

PCAs_results = {
    "train": {},
    "valid": {},
}

LDAs_results = {
    "train": {},
    "valid": {},
}

TSNEs_results = {
    "train": {},
    "valid": {},
}

for n_components in n_components_to_test["PCA"]:
    print(f'[PCA] Extracting features (# components:{n_components})')
    
    PCAs_results["train"][n_components] = []
    PCAs_results["valid"][n_components] = []

    PCA_instance = PCA(n_components=n_components)
    PCA_instance.fit(dataset["train"]['data'])

    PCAs_results["train"][n_components] = PCA_instance.transform(dataset["train"]['data'])
    PCAs_results["valid"][n_components] = PCA_instance.transform(dataset["valid"]['data'])

    PCAs_instances[n_components] = PCA_instance
    
    results_PCA.append(
        {
            "METHOD": "PCA",
            "# Components": n_components,
            "CHANNEL": "Grayscale",
            "Explained Variance Ratio": np.sum(
                PCA_instance.explained_variance_ratio_, axis=0
            ),
        }
    )

for n_components in n_components_to_test["LDA"]:
    print(f'[LDA] Extracting features (# components:{n_components})')
    
    LDAs_instances[n_components] = []

    LDAs_results["train"][n_components] = []
    LDAs_results["valid"][n_components] = []

    LDA_instance = LinearDiscriminantAnalysis(n_components=n_components)

    LDA_instance.fit(dataset["train"]['data'], dataset["train"]["labels"])

    LDAs_results["train"][n_components] = LDA_instance.transform(dataset["train"]['data'])
    LDAs_results["valid"][n_components] = LDA_instance.transform(dataset["valid"]['data'])

    LDAs_instances[n_components] = LDA_instance

    results_LDA.append(
        {
            "METHOD": "LDA",
            "# Components": n_components,
            "CHANNEL": "Grayscale",
            "Explained Variance Ratio": np.sum(
                LDA_instance.explained_variance_ratio_, axis=0
            ),
        }
    )

for n_components in n_components_to_test["TSNE"]:
    print(f'[t-SNE] Extracting features (# components:{n_components})')

    TSNEs_instances[n_components] = []

    TSNEs_results["train"][n_components] = []

    TSNE_instance_train = TSNE(n_components=n_components, verbose=1, n_iter=3000)
    TSNE_instance_valid = TSNE(n_components=n_components, verbose=1, n_iter=3000)

    TSNEs_results["train"][n_components] = TSNE_instance_train.fit_transform(
        LDAs_results["train"][7]
    )
    TSNEs_results["valid"][n_components] = TSNE_instance_valid.fit_transform(
        LDAs_results["valid"][7]
    )

    TSNEs_instances[n_components] = [TSNE_instance_train, TSNE_instance_valid]


# Pandas tables
df_results_PCA = pd.DataFrame(results_PCA)
df_results_LDA = pd.DataFrame(results_LDA)

def highlight_cells(val):
    color = ""
    if val > 0.80:
        color = "background-color: lightgreen; color: black; font-weight: bold"
    elif val < 0.80:
        color = "background-color: lightcoral; color: black; font-weight: bold"
    return color

# Apply the style
df_results_PCA_styled = (
    df_results_PCA.style.map(highlight_cells, subset=["Explained Variance Ratio"])
    .set_caption("PCA Results")
    .set_properties(**{"text-align": "center"})
)

df_results_LDA_styled = (
    df_results_LDA.style.map(highlight_cells, subset=["Explained Variance Ratio"])
    .set_caption("LDA Results")
    .set_properties(**{"text-align": "center"})
)

os.makedirs('.pkl', exist_ok=True)

pickle.dump(PCAs_results, open(".pkl/classic_pca_out.pkl", "wb"))
pickle.dump(LDAs_results, open(".pkl/classic_lda_out.pkl", "wb"))
pickle.dump(TSNEs_results, open(".pkl/classic_tsne_out.pkl", "wb"))

display(df_results_PCA_styled)
display(df_results_LDA_styled)

[PCA] Extracting features (# components:3)
[PCA] Extracting features (# components:10)
[PCA] Extracting features (# components:50)
[PCA] Extracting features (# components:100)
[PCA] Extracting features (# components:200)
[PCA] Extracting features (# components:500)
[PCA] Extracting features (# components:1200)
[LDA] Extracting features (# components:3)
[LDA] Extracting features (# components:5)
[LDA] Extracting features (# components:7)
[LDA] Extracting features (# components:9)
[t-SNE] Extracting features (# components:2)
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1500 samples in 0.006s...
[t-SNE] Computed neighbors for 1500 samples in 0.067s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1500
[t-SNE] Computed conditional probabilities for sample 1500 / 1500
[t-SNE] Mean sigma: 0.809948
[t-SNE] KL divergence after 250 iterations with early exaggeration: 61.021358
[t-SNE] KL divergence after 3000 iterations: 0.907271
[t-SNE] Computing 91 nearest neighbor

,METHOD,# Components,CHANNEL,Explained Variance Ratio
0,PCA,3,Grayscale,0.346227
1,PCA,10,Grayscale,0.523715
2,PCA,50,Grayscale,0.735099
3,PCA,100,Grayscale,0.806670
4,PCA,200,Grayscale,0.869063
5,PCA,500,Grayscale,0.941782
6,PCA,1200,Grayscale,0.993520


,METHOD,# Components,CHANNEL,Explained Variance Ratio
0,LDA,3,Grayscale,0.575544
1,LDA,5,Grayscale,0.742640
2,LDA,7,Grayscale,0.883926
3,LDA,9,Grayscale,1.000000


### 2D/3D Data visualization

In [5]:
 
PCAs_results = pickle.load(open(".pkl/classic_pca_out.pkl", "rb"))
LDAs_results = pickle.load(open(".pkl/classic_lda_out.pkl", "rb"))
TSNEs_results = pickle.load(open(".pkl/classic_tsne_out.pkl", "rb"))

tSNE_fig_2D = plt.figure()
tSNE_3D = tSNE_fig_2D.add_subplot()

for i in range(len(dataset["train"]["unique_labels"])):
    classIdxs = dataset["train"]["labels"] == i

    tsne_features = TSNEs_results["train"][2][classIdxs, :]

    tSNE_3D.set_label(dataset["train"]["unique_labels"][i])
    tSNE_3D.scatter(
        tsne_features[:, 0],
        tsne_features[:, 1],
        marker=".",
        label=dataset["train"]["unique_labels"][i],
    )

plt.legend(loc="upper left")

# 3D plot
tSNE_fig_3D = plt.figure()
tSNE_3D = tSNE_fig_3D.add_subplot(projection="3d")

for i in range(len(dataset["train"]["unique_labels"])):
    classIdxs = dataset["train"]["labels"] == i

    tsne_features = TSNEs_results["train"][3][classIdxs, :]

    tSNE_3D.scatter(
        tsne_features[:, 0],
        tsne_features[:, 1],
        tsne_features[:, 2],
        marker=".",
        label=dataset["train"]["unique_labels"][i],
    )

plt.legend(loc="upper left")

### Classification - KNN

In [6]:
PCAs_results = pickle.load(open(".pkl/classic_pca_out.pkl", "rb"))
LDAs_results = pickle.load(open(".pkl/classic_lda_out.pkl", "rb"))
TSNEs_results = pickle.load(open(".pkl/classic_tsne_out.pkl", "rb"))

# Number of neighbors to test
k_to_test = {
    "PCA": [3, 5, 9, 15, 21, 55, 111, 251],
    "LDA": [3, 5, 9, 15, 21, 55, 111, 251],
}

KNN_PCA_stats = []
KNN_LDA_stats = []

for k_idx, k in enumerate(k_to_test["PCA"]):

    KNN_PCA_stats.insert(k_idx, [k])

    for n_components_idx, n_components in enumerate(n_components_to_test["PCA"]):
        knn = OneVsOneClassifier(KNeighborsClassifier(k))

        knn.fit(PCAs_results["train"][n_components], dataset["train"]["labels"])
        preds = knn.predict(PCAs_results["valid"][n_components])

        accuracy = round(accuracy_score(dataset["valid"]["labels"], preds), 3)
        f_one_score = round(f1_score(dataset["valid"]["labels"], preds, average="macro"), 3)

        KNN_PCA_stats[k_idx].insert(n_components_idx + 1, (accuracy, f_one_score))

        # ConfusionMatrixDisplay(confusion_matrix(dataset['valid']['labels'], preds),display_labels=dataset['valid']['unique_labels']).plot()

KNN_PCA_df = pd.DataFrame(
    KNN_PCA_stats, columns=["k\\PCA components"] + n_components_to_test["PCA"]
)
display(KNN_PCA_df)

for k_idx, k in enumerate(k_to_test["LDA"]):

    KNN_LDA_stats.insert(k_idx, [k])

    for n_components_idx, n_components in enumerate(n_components_to_test["LDA"]):
        knn = OneVsOneClassifier(KNeighborsClassifier(k))

        knn.fit(LDAs_results["train"][n_components], dataset["train"]["labels"])
        preds = knn.predict(LDAs_results["valid"][n_components])

        accuracy = round(accuracy_score(dataset["valid"]["labels"], preds), 3)
        f_one_score = round(f1_score(dataset["valid"]["labels"], preds, average="macro"), 3)

        KNN_LDA_stats[k_idx].insert(n_components_idx + 1, (accuracy, f_one_score))
        # ConfusionMatrixDisplay(confusion_matrix(dataset['valid']['labels'], preds),display_labels=dataset['valid']['unique_labels']).plot()


KNN_LDA_df = pd.DataFrame(
    KNN_LDA_stats, columns=["k\\LDA components"] + n_components_to_test["LDA"]
)
display(KNN_LDA_df)


,k\PCA components,3,10,50,100,200,500,1200
0,3,"(0.162, 0.159)","(0.178, 0.177)","(0.176, 0.177)","(0.158, 0.155)","(0.156, 0.15)","(0.148, 0.14)","(0.144, 0.128)"
1,5,"(0.16, 0.157)","(0.166, 0.164)","(0.146, 0.136)","(0.146, 0.143)","(0.138, 0.133)","(0.14, 0.133)","(0.13, 0.125)"
2,9,"(0.172, 0.164)","(0.184, 0.181)","(0.162, 0.156)","(0.146, 0.134)","(0.134, 0.126)","(0.134, 0.13)","(0.126, 0.113)"
3,15,"(0.2, 0.193)","(0.192, 0.177)","(0.154, 0.14)","(0.164, 0.149)","(0.148, 0.13)","(0.146, 0.123)","(0.138, 0.111)"
4,21,"(0.182, 0.172)","(0.202, 0.182)","(0.166, 0.141)","(0.154, 0.132)","(0.156, 0.133)","(0.138, 0.106)","(0.15, 0.123)"
5,55,"(0.184, 0.162)","(0.216, 0.19)","(0.18, 0.149)","(0.176, 0.144)","(0.166, 0.133)","(0.164, 0.122)","(0.17, 0.123)"
6,111,"(0.206, 0.172)","(0.238, 0.202)","(0.19, 0.163)","(0.18, 0.147)","(0.172, 0.134)","(0.162, 0.112)","(0.158, 0.103)"
7,251,"(0.192, 0.138)","(0.196, 0.133)","(0.196, 0.138)","(0.198, 0.142)","(0.194, 0.138)","(0.194, 0.14)","(0.184, 0.132)"


,k\LDA components,3,5,7,9
0,3,"(0.136, 0.118)","(0.14, 0.13)","(0.16, 0.151)","(0.162, 0.154)"
1,5,"(0.134, 0.116)","(0.144, 0.135)","(0.176, 0.166)","(0.156, 0.148)"
2,9,"(0.136, 0.117)","(0.164, 0.155)","(0.166, 0.154)","(0.154, 0.147)"
3,15,"(0.132, 0.112)","(0.148, 0.139)","(0.166, 0.156)","(0.156, 0.15)"
4,21,"(0.122, 0.107)","(0.152, 0.144)","(0.16, 0.152)","(0.16, 0.153)"
5,55,"(0.13, 0.117)","(0.146, 0.137)","(0.166, 0.155)","(0.154, 0.147)"
6,111,"(0.122, 0.11)","(0.144, 0.135)","(0.166, 0.154)","(0.15, 0.144)"
7,251,"(0.12, 0.105)","(0.14, 0.132)","(0.16, 0.15)","(0.156, 0.151)"


### Classification - SVM

In [7]:

PCAs_results = pickle.load(open(".pkl/classic_pca_out.pkl", "rb"))
LDAs_results = pickle.load(open(".pkl/classic_lda_out.pkl", "rb"))
TSNEs_results = pickle.load(open(".pkl/classic_tsne_out.pkl", "rb"))

kernels_to_test = {
    "PCA": ["linear", "poly", "sigmoid"], 
    "LDA": ["linear", "poly", "sigmoid"], 
}

SVM_PCA_stats = []
SVM_LDA_stats = []

for kernel_idx, kernel in enumerate(kernels_to_test["PCA"]):

    SVM_PCA_stats.insert(kernel_idx, [kernel])

    for n_components_idx, n_components in enumerate(n_components_to_test["PCA"]):
        svm = OneVsOneClassifier(SVC(kernel=kernel))

        svm.fit(PCAs_results["train"][n_components], dataset["train"]["labels"])

        preds = svm.predict(PCAs_results["valid"][n_components])

        accuracy = round(accuracy_score(dataset["valid"]["labels"], preds), 3)
        f_one_score = round(f1_score(dataset["valid"]["labels"], preds, average="weighted"), 3)

        SVM_PCA_stats[kernel_idx].insert(n_components_idx + 1, (accuracy, f_one_score))

SVM_PCA_df = pd.DataFrame(
    SVM_PCA_stats, columns=["kernel\\PCA components"] + n_components_to_test["PCA"]
)
display(SVM_PCA_df)

for kernel_idx, kernel in enumerate(kernels_to_test["LDA"]):

    SVM_LDA_stats.insert(kernel_idx, [kernel])

    for n_components_idx, n_components in enumerate(n_components_to_test["LDA"]):
        svm = OneVsOneClassifier(SVC(kernel=kernel))

        svm.fit(LDAs_results["train"][n_components], dataset["train"]["labels"])

        preds = svm.predict(LDAs_results["valid"][n_components])

        accuracy = round(accuracy_score(dataset["valid"]["labels"], preds), 3)
        f_one_score = round(f1_score(dataset["valid"]["labels"], preds, average="weighted"), 3)

        SVM_LDA_stats[kernel_idx].insert(n_components_idx + 1, (accuracy, f_one_score))

SVM_LDA_df = pd.DataFrame(
    SVM_LDA_stats, columns=["kernel\\LDA components"] + n_components_to_test["LDA"]
)
display(SVM_LDA_df)


,kernel\PCA components,3,10,50,100,200,500,1200
0,linear,"(0.196, 0.165)","(0.232, 0.209)","(0.192, 0.185)","(0.168, 0.164)","(0.184, 0.183)","(0.182, 0.182)","(0.198, 0.195)"
1,poly,"(0.166, 0.166)","(0.21, 0.199)","(0.212, 0.209)","(0.208, 0.206)","(0.204, 0.205)","(0.202, 0.204)","(0.206, 0.208)"
2,sigmoid,"(0.138, 0.112)","(0.182, 0.158)","(0.198, 0.181)","(0.224, 0.205)","(0.226, 0.207)","(0.238, 0.22)","(0.238, 0.22)"


,kernel\LDA components,3,5,7,9
0,linear,"(0.12, 0.097)","(0.134, 0.124)","(0.136, 0.124)","(0.146, 0.136)"
1,poly,"(0.128, 0.104)","(0.126, 0.111)","(0.158, 0.141)","(0.13, 0.121)"
2,sigmoid,"(0.138, 0.126)","(0.142, 0.137)","(0.15, 0.143)","(0.148, 0.135)"


### Classification - SGD

In [8]:

PCAs_results = pickle.load(open(".pkl/classic_pca_out.pkl", "rb"))
LDAs_results = pickle.load(open(".pkl/classic_lda_out.pkl", "rb"))
TSNEs_results = pickle.load(open(".pkl/classic_tsne_out.pkl", "rb"))

losses_to_test = {
    "PCA": ["modified_huber", "log_loss", "hinge"],
    "LDA": ["modified_huber", "log_loss", "hinge"]
}

SGD_PCA_stats = []
SGD_LDA_stats = []

for loss_idx, loss in enumerate(losses_to_test["PCA"]):

    SGD_PCA_stats.insert(loss_idx, [loss])

    for n_components_idx, n_components in enumerate(n_components_to_test["PCA"]):
        sgd = SGDClassifier(loss=loss, max_iter=10000)

        sgd.fit(PCAs_results["train"][n_components], dataset["train"]["labels"])

        preds = sgd.predict(PCAs_results["valid"][n_components])

        accuracy = round(accuracy_score(dataset["valid"]["labels"], preds), 3)
        f_one_score = round(f1_score(dataset["valid"]["labels"], preds, average="macro"), 3)

        SGD_PCA_stats[loss_idx].insert(
            n_components_idx + 1, (accuracy, f_one_score)
        )

SGD_PCA_df = pd.DataFrame(
    SGD_PCA_stats, columns=["loss\\PCA"] + n_components_to_test["PCA"]
)
display(SGD_PCA_df)

for loss_idx, loss in enumerate(losses_to_test["LDA"]):

    SGD_LDA_stats.insert(loss_idx, [loss])

    for n_components_idx, n_components in enumerate(n_components_to_test["LDA"]):
        sgd = SGDClassifier(loss=loss, max_iter=10000)

        sgd.fit(LDAs_results["train"][n_components], dataset["train"]["labels"])

        preds = sgd.predict(LDAs_results["valid"][n_components])

        accuracy = round(accuracy_score(dataset["valid"]["labels"], preds), 3)
        f_one_score = round(f1_score(dataset["valid"]["labels"], preds, average="macro"), 3)

        SGD_LDA_stats[loss_idx].insert(
            n_components_idx + 1, (accuracy, f_one_score)
        )

SGD_LDA_df = pd.DataFrame(
    SGD_LDA_stats, columns=["loss\\LDA"] + n_components_to_test["LDA"]
)
display(SGD_LDA_df)

,loss\PCA,3,10,50,100,200,500,1200
0,modified_huber,"(0.162, 0.124)","(0.162, 0.138)","(0.168, 0.151)","(0.194, 0.187)","(0.164, 0.155)","(0.182, 0.181)","(0.152, 0.148)"
1,log_loss,"(0.148, 0.092)","(0.186, 0.176)","(0.184, 0.174)","(0.156, 0.149)","(0.198, 0.19)","(0.162, 0.159)","(0.16, 0.155)"
2,hinge,"(0.144, 0.128)","(0.158, 0.134)","(0.19, 0.173)","(0.16, 0.151)","(0.186, 0.174)","(0.174, 0.172)","(0.162, 0.16)"


,loss\LDA,3,5,7,9
0,modified_huber,"(0.094, 0.075)","(0.13, 0.113)","(0.134, 0.119)","(0.146, 0.139)"
1,log_loss,"(0.134, 0.103)","(0.122, 0.114)","(0.146, 0.131)","(0.158, 0.144)"
2,hinge,"(0.132, 0.111)","(0.148, 0.133)","(0.152, 0.13)","(0.134, 0.117)"


### Classification - K-Means

In [9]:

PCAs_results = pickle.load(open(".pkl/classic_pca_out.pkl", "rb"))
LDAs_results = pickle.load(open(".pkl/classic_lda_out.pkl", "rb"))
TSNEs_results = pickle.load(open(".pkl/classic_tsne_out.pkl", "rb"))

algos_to_test = {
    "PCA": ["lloyd", "elkan"],
    "LDA": ["lloyd", "elkan"]
}

KMEANS_PCA_stats = []
KMEANS_LDA_stats = []

for algo_idx, algorithm in enumerate(algos_to_test["PCA"]):

    KMEANS_PCA_stats.insert(algo_idx, [algorithm])

    for n_components_idx, n_components in enumerate(n_components_to_test["PCA"]):
        kmeans = KMeans(algorithm=algorithm, n_init='auto')

        kmeans.fit(PCAs_results["train"][n_components])

        preds = kmeans.predict(PCAs_results["valid"][n_components])

        score = round(adjusted_rand_score(dataset["valid"]["labels"], preds), 3)

        KMEANS_PCA_stats[algo_idx].insert(n_components_idx + 1, score)

KMEANS_PCA_df = pd.DataFrame(
    KMEANS_PCA_stats, columns=["algorithm\\PCA"] + n_components_to_test["PCA"]
)
display(KMEANS_PCA_df)

for algo_idx, algorithm in enumerate(algos_to_test["LDA"]):

    KMEANS_LDA_stats.insert(algo_idx, [algorithm])

    for n_components_idx, n_components in enumerate(n_components_to_test["LDA"]):
        kmeans = KMeans(algorithm=algorithm, n_init='auto')

        kmeans.fit(LDAs_results["train"][n_components])

        preds = kmeans.predict(LDAs_results["valid"][n_components])

        score = round(adjusted_rand_score(dataset["valid"]["labels"], preds), 3)

        KMEANS_LDA_stats[algo_idx].insert(n_components_idx + 1, score)

KMEANS_LDA_df = pd.DataFrame(
    KMEANS_LDA_stats, columns=["algorithm\\LDA"] + n_components_to_test["LDA"]
)

display(KMEANS_LDA_df)

,algorithm\PCA,3,10,50,100,200,500,1200
0,lloyd,0.016,0.020,0.024,0.028,0.024,0.020,0.033
1,elkan,0.027,0.022,0.025,0.019,0.023,0.021,0.023


,algorithm\LDA,3,5,7,9
0,lloyd,0.006,0.004,0.013,0.007
1,elkan,0.006,0.009,0.017,0.002
